# Doc2Vecの概要

こちらを参考にしました。

http://radimrehurek.com/gensim/models/doc2vec.html

実際のPython環境における実行例は、<a href="11-doc2vec-tutor.ipynb"><b>こちらのレポート</b></a>をご参照いただければ幸いです。

## (1) 学習セット

gensimのDoc2Vecにおける学習セットのレコードは以下のような形式となります。

（コーパスのリストと、ラベルからなるリストが、学習セット１サンプル分となります）

```
['Ｗｉｎｄｏｗｓ', 'ログイン', '出来る', 'ない'], [4420]
['ログイン', '出来る', 'ない'], [4420]
['パソコン', 'ログイン', '出来る', 'ない'], [4420]
：
```

ラベルの重複は可能です（同一ラベルに複数のサンプルを許容している様子）。

ただし、重複するとモデル内で上書きされるようです。

重複しないようにするためには、例えば下記のようにラベルにユニークなIDを付与するようにします。

```
['Ｗｉｎｄｏｗｓ', 'ログイン', '出来る', 'ない'], ['4420_1']
['ログイン', '出来る', 'ない'], ['4420_2']
['パソコン', 'ログイン', '出来る', 'ない'], ['4420_3']
：
```

## (2) 学習処理

学習セットをクラス「gensim.models.doc2vec.Doc2Vec」に渡して学習処理を実行させます。

学習が終わると、下記のようなイメージの配列（学習時のパラメータとして指定したfeature数分の重み値配列）が作成されます。

この重み値配列が、学習セットに付与したラベル数分、作成され、「gensim.models.doc2vec.DocvecsArray」というクラスのインスタンスに収容されます。

http://radimrehurek.com/gensim/models/doc2vec.html#gensim.models.doc2vec.DocvecsArray

```
[ 0.03135058,  0.06804109, -0.06892224,  0.91631955, -0.90788156,
 -1.27675772,  0.12107785, -0.49540758,  0.32016867, -0.59504598,
  0.02170568, -0.31601551, -0.9805398 ,  0.09150546, -0.84680676,
  ：
 -0.60410583, -0.31215325, -0.64900941, -0.58420479, -0.13754645,
  0.02447186, -0.57329637, -0.78971535,  0.53362709, -0.23328291] <---要素はfeature数分
```

ただし、Doc2Vecにおけるfeatureは、scikit-learnのベクトライザーのように、ボキャブラリの単語と対応しているわけではないようです。

何らかの演算（次元削除手法）により、例えば数百件の単語でできた文書の特徴が、学習時にパラメータとして指定したfeature（通常は100前後を指定）に集約される動きになっています。

## (3) 予測処理

予測を実行するときは、下記のようなコーパスのリストを使用します。

```
['情報','システム','アドレス']
```

上記コーパスから、モデルのinfer_vector関数を使用し、近似ベクトルを生成します。

こちらは下記のようなイメージの、numpy arrayオブジェクトです。

```
[  4.38769273e-02,   3.98379229e-02,  -1.61702514e-01,
   4.28867340e-02,  -4.16987725e-02,  -1.88497797e-01,
   1.46046523e-02,  -9.24882218e-02,  -5.91080869e-03,
  ：
  -1.14380161e-03,   1.09144533e-02,   3.87810133e-02,
  -2.19986439e-02,  -6.48149401e-02,  -8.24136101e-03,
   4.17991243e-02,   1.62045151e-01] <---要素はfeature数分
```

モデルで生成された、DocvecsArrayインスタンスのmost_similar関数を使用し、上記ベクトルに近似しているベクトルを、モデルの中から探して１０件出力します。

（出力件数＝１０件はデフォルト）

(ラベル, 類似度) の形式で出力されます。

```
[(7040, 0.6151561141014099),
 (4621, 0.6114732027053833),
 (7068, 0.5491945147514343),
 (4563, 0.5432988405227661),
 (4568, 0.5397956967353821),
 (4524, 0.5377195477485657),
 (4566, 0.5245414972305298),
 (4498, 0.5226050019264221),
 (4460, 0.516629695892334),
 (4602, 0.5153311491012573)]
```

なお、most_similar関数による、近似ベクトルの探索は、コサイン類似度をつかって行われるようです。

　http://radimrehurek.com/gensim/models/doc2vec.html#gensim.models.doc2vec.DocvecsArray.most_similar　より引用：

　This method computes cosine similarity between a simple mean of the projection weight vectors of the given docs